In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)


features = ['e_lang_pt', 'faq2', 'ADAS_Q1', 'LDELTOTAL', 'faq3', 'ADAS_Q4', 'tmab_time', 'faq9', 'e_divatt_cg', 'ADAS_Q8', 'faq7', 'e_lang_cg', 'ADAS_Q7', 'faq10', 'e_memory_pt', 'CATANIMSC', 'e_memory_cg']

categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])
data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']

In [2]:
_ = features
print(len(_))
print('[%s]' % ', '.join(map(str, _)))

17
[e_lang_pt, faq2, ADAS_Q1, LDELTOTAL, faq3, ADAS_Q4, tmab_time, faq9, e_divatt_cg, ADAS_Q8, faq7, e_lang_cg, ADAS_Q7, faq10, e_memory_pt, CATANIMSC, e_memory_cg]


In [11]:
# Divide data in half for tuning, to reduce leakage when using split/repeat cross-validation
X, X_test, y, y_test = train_test_split(X, y, test_size=0.5, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

In [20]:
y_.value_counts()

1.0    622
0.0    574
2.0     75
Name: CDGLOBAL, dtype: int64

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3)
df.describe()

Wall time: 4.19 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.838068,0.930093,0.888776,0.884080,0.886979,0.818000,0.851786,0.852490,0.670833,0.982065,0.963010,0.826449,0.798627,0.910053,0.901190,0.944099
std,0.037213,0.012558,0.020648,0.022045,0.029709,0.027644,0.016427,0.016456,0.201375,0.009674,0.006800,0.096657,0.052160,0.009601,0.010951,0.009664
min,0.761364,0.907407,0.846939,0.839015,0.854167,0.760000,0.816327,0.817500,0.333333,0.961957,0.954082,0.663949,0.707702,0.886787,0.877551,0.926623
25%,0.823864,0.923611,0.876276,0.871423,0.854167,0.797500,0.845663,0.845833,0.562500,0.976902,0.959184,0.772418,0.770439,0.906530,0.897109,0.937662
50%,0.840909,0.925926,0.895408,0.891414,0.880208,0.820000,0.852041,0.852292,0.666667,0.983696,0.964286,0.826540,0.795612,0.912323,0.901361,0.946545
75%,0.866477,0.935185,0.904337,0.901094,0.908854,0.840000,0.862245,0.863750,0.833333,0.990489,0.965561,0.906476,0.839725,0.914975,0.908163,0.952311
max,0.897727,0.953704,0.918367,0.916456,0.937500,0.860000,0.882653,0.883125,0.916667,0.994565,0.974490,0.947464,0.870581,0.928105,0.921769,0.955899


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  5.8min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=0.5,
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                  

In [7]:
rf_search_rand.best_params_

{'n_estimators': 1200,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 0.7,
 'max_depth': 70,
 'bootstrap': True}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.833333,0.932099,0.887755,0.882716,0.891204,0.816667,0.853175,0.853935,0.694444,0.983092,0.965420,0.838768,0.806327,0.910619,0.902116,0.947007
std,0.026650,0.017930,0.020567,0.021025,0.035749,0.016583,0.018356,0.018628,0.150231,0.008782,0.003401,0.071052,0.032604,0.009830,0.012238,0.011411
min,0.795455,0.907407,0.862245,0.856061,0.854167,0.800000,0.831633,0.832083,0.500000,0.972826,0.959184,0.747283,0.759470,0.898309,0.887755,0.931556
25%,0.806818,0.916667,0.862245,0.857113,0.864583,0.800000,0.831633,0.832292,0.500000,0.978261,0.964286,0.747283,0.767045,0.898556,0.887755,0.932959
50%,0.840909,0.925926,0.892857,0.889099,0.875000,0.810000,0.857143,0.857500,0.750000,0.978261,0.964286,0.864130,0.825758,0.914729,0.904762,0.951638
75%,0.852273,0.953704,0.903061,0.897306,0.937500,0.830000,0.867347,0.868750,0.833333,0.994565,0.969388,0.905797,0.831124,0.917815,0.911565,0.956918
max,0.863636,0.953704,0.913265,0.908670,0.937500,0.840000,0.877551,0.878750,0.833333,0.994565,0.969388,0.905797,0.831439,0.922756,0.918367,0.957252


## Grid Search

In [11]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [1100, 1200, 1300],
               'max_features': [.6, .7, .8],
               'max_depth': [60, 70, 80],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  8.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=0.5,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=210,
                                    

In [12]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 0.6,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 1100}

In [13]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.823232,0.938272,0.886621,0.880752,0.898148,0.807778,0.852041,0.852963,0.694444,0.983092,0.965420,0.838768,0.805275,0.909714,0.901361,0.947349
std,0.018939,0.012249,0.013201,0.013571,0.029819,0.018559,0.010823,0.011068,0.150231,0.008782,0.003401,0.071052,0.037865,0.005555,0.007215,0.011450
min,0.795455,0.925926,0.867347,0.860690,0.875000,0.780000,0.836735,0.837500,0.500000,0.972826,0.959184,0.747283,0.751894,0.901396,0.891156,0.931937
25%,0.806818,0.925926,0.872449,0.866372,0.875000,0.800000,0.841837,0.842500,0.500000,0.978261,0.964286,0.747283,0.759470,0.904729,0.894558,0.933748
50%,0.818182,0.935185,0.892857,0.885943,0.885417,0.810000,0.852041,0.852500,0.750000,0.978261,0.964286,0.864130,0.821970,0.909423,0.901361,0.951563
75%,0.840909,0.953704,0.897959,0.891625,0.937500,0.810000,0.862245,0.862708,0.833333,0.994565,0.969388,0.905797,0.834596,0.912756,0.908163,0.957597
max,0.852273,0.953704,0.903061,0.897306,0.937500,0.840000,0.867347,0.868750,0.833333,0.994565,0.969388,0.905797,0.838384,0.917815,0.911565,0.958277


In [3]:
rf = RandomForestClassifier(n_estimators=1100, max_features=.6, max_depth = 60, min_samples_split = 4, min_samples_leaf = 4, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3, imbalanced='over', avg_strategy='weighted')
df.describe()

Wall time: 51.7 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.835795,0.933796,0.889796,0.884796,0.879167,0.829000,0.853571,0.854083,0.779167,0.975815,0.963776,0.877491,0.853571,0.851826,0.853571,0.947663
std,0.029833,0.014801,0.019753,0.020568,0.025155,0.017442,0.018735,0.018823,0.090785,0.005973,0.008899,0.046289,0.018735,0.017756,0.018735,0.009919
min,0.784091,0.907407,0.852041,0.845749,0.843750,0.790000,0.821429,0.821875,0.666667,0.967391,0.948980,0.817029,0.821429,0.820835,0.821429,0.932862
25%,0.821023,0.923611,0.885204,0.879261,0.861979,0.817500,0.843112,0.843490,0.729167,0.972826,0.961735,0.853034,0.843112,0.840922,0.843112,0.941295
50%,0.846591,0.935185,0.897959,0.893729,0.875000,0.840000,0.857143,0.857500,0.750000,0.975543,0.964286,0.865489,0.857143,0.854567,0.857143,0.950235
75%,0.852273,0.944444,0.903061,0.897569,0.898438,0.840000,0.863520,0.864323,0.833333,0.979620,0.969388,0.908514,0.863520,0.861203,0.863520,0.956301
max,0.875000,0.953704,0.908163,0.905093,0.916667,0.850000,0.877551,0.878333,0.916667,0.983696,0.974490,0.944746,0.877551,0.875947,0.877551,0.958520


In [2]:
# All samples
rf = RandomForestClassifier(n_estimators=1100, max_features=.6, max_depth = 60, min_samples_split = 4, min_samples_leaf = 4, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), num_classes=3, imbalanced='over', avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 35min 54s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.864058,0.923369,0.897143,0.893713,0.875187,0.850017,0.862602,0.862602,0.742647,0.979134,0.965459,0.860890,0.862602,0.860476,0.862602,0.946450
std,0.027962,0.017910,0.014818,0.015560,0.025364,0.028277,0.019865,0.019865,0.105759,0.006841,0.010008,0.053901,0.019865,0.020643,0.019865,0.010078
min,0.792308,0.890244,0.857143,0.850422,0.816327,0.782313,0.806122,0.806122,0.470588,0.960289,0.931973,0.715439,0.806122,0.800570,0.806122,0.931303
25%,0.844231,0.908537,0.887755,0.885038,0.857143,0.829932,0.850340,0.850340,0.647059,0.974729,0.959184,0.814504,0.850340,0.848457,0.850340,0.936459
50%,0.869231,0.920732,0.897959,0.895779,0.884354,0.850340,0.860544,0.860544,0.764706,0.981949,0.969388,0.871523,0.860544,0.859546,0.860544,0.947355
75%,0.884615,0.939024,0.911565,0.907176,0.897959,0.877551,0.877551,0.877551,0.823529,0.981949,0.972789,0.902739,0.877551,0.874457,0.877551,0.953536
max,0.915385,0.963415,0.921769,0.920310,0.918367,0.891156,0.897959,0.897959,1.000000,0.992780,0.979592,0.985560,0.897959,0.896440,0.897959,0.968358


# Outputs

In [3]:
### Save outputs ###
q = 'CDR_MultiC_BorutaSHAP_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)